# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [ ]:
%pip install xlsxwriter

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

/tmp/ipykernel_2267/825764589.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol}?token={IEX_CLOUD_API_TOKEN}'
print(api_url)
data = requests.get(api_url).json()
data[0]

https://api.iex.cloud/v1/data/core/QUOTE/AAPL?token=sk_7e3b1d2506bb4838a09e400b322b4904


{'avgTotalVolume': 54749865,
 'calculationPrice': 'close',
 'change': 2.05,
 'changePercent': 0.01124,
 'close': 184.37,
 'closeSource': 'official',
 'closeTime': 1708635600180,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 184.32,
 'delayedPriceTime': 1708635594024,
 'extendedChange': -0.37,
 'extendedChangePercent': -0.00201,
 'extendedPrice': 184,
 'extendedPriceTime': 1708649996941,
 'high': 184.955,
 'highSource': '15 minute delayed price',
 'highTime': 1708635599015,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 184.34,
 'iexCloseTime': 1708635599410,
 'iexLastUpdated': 1708635599410,
 'iexMarketPercent': 0.016965127959408407,
 'iexOpen': 183.59,
 'iexOpenTime': 1708612200340,
 'iexRealtimePrice': 184.34,
 'iexRealtimeSize': 100,
 'iexVolume': 887144,
 'lastTradeTime': 1708635599410,
 'latestPrice': 184.37,
 'latestSource': 'Close',
 'latestTime': 'February 22, 2024',
 'latestUpdate': 1708635600180,
 'latestVolume': 522

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data[0]['latestPrice']
market_cap = data[0]['marketCap']

price, market_cap

(184.37, 2847019599970)

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalisation', 'Number of Shares to Buy']
df = pd.DataFrame(columns = my_columns)
df

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy


In [7]:
new_df = pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
)

df = pd.concat([df, new_df.to_frame().T], ignore_index=True)

df

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,AAPL,184.37,2847019599970,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
df = pd.DataFrame(columns = my_columns)
# for stock in stocks['Ticker'][:5]:
for stock in stocks['Ticker']:
    api_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{stock}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()

    # for data_arr in data:
            
    new_df = pd.Series(
        [
            stock,
            data[0]['latestPrice'],
            data[0]['marketCap'],
            'N/A'
        ],
        index = my_columns
    )

    df = pd.concat([df, new_df.to_frame().T], ignore_index=True)

df

# 6m 41.8s

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,132.02,38687102118,N/A
1,AAL,15.24,9959957982,N/A
2,AAP,60.69,3611175773,N/A
3,AAPL,184.37,2847019599970,N/A
4,ABBV,176.75,312224166203,N/A
...,...,...,...,...
500,YUM,137.33,38635911332,N/A
501,ZBH,128.54,26862380592,N/A
502,ZBRA,279.48,14360076187,N/A
503,ZION,39.65,5874099880,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [9]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [10]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

df = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    # for data_arr in data[:1]:
    for data_arr in data:
            
        new_df = pd.Series(
            [
                data_arr['symbol'],
                data_arr['latestPrice'],
                data_arr['marketCap'],
                'N/A'
            ],
            index = my_columns
        )

        df = pd.concat([df, new_df.to_frame().T], ignore_index=True)

df

# 8.1s

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,132.02,38687102118,N/A
1,AAL,15.24,9959957982,N/A
2,AAP,60.69,3611175773,N/A
3,AAPL,184.37,2847019599970,N/A
4,ABBV,176.75,312224166203,N/A
...,...,...,...,...
500,YUM,137.33,38635911332,N/A
501,ZBH,128.54,26862380592,N/A
502,ZBRA,279.48,14360076187,N/A
503,ZION,39.65,5874099880,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [11]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
    print(val)

except ValueError:
    print('Please try again. \n')

10000000.0


In [12]:
position_size = val/len(df.index)
# print(position_size)

for i in range(0, len(df.index)):
    # print(i)
    df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/df.loc[i, 'Stock Price'])

df

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,132.02,38687102118,149
1,AAL,15.24,9959957982,1299
2,AAP,60.69,3611175773,326
3,AAPL,184.37,2847019599970,107
4,ABBV,176.75,312224166203,112
...,...,...,...,...
500,YUM,137.33,38635911332,144
501,ZBH,128.54,26862380592,154
502,ZBRA,279.48,14360076187,70
503,ZION,39.65,5874099880,499


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [13]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [14]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [15]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalisation', dollar_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [16]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalisation', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [17]:
writer.close()